In [1]:
import numpy as np
import graphinglib as gl
import pyregion

from src.statistics.advanced_stats import structure_function
from src.hdu.maps.map import Map

In [2]:
def get_plottables(data: np.ndarray, region_radius: float) -> list[gl.Curve]:
    sorted_data = data[np.argsort(data[:,0])]
    curve = gl.Curve(
        np.log(sorted_data[:,0]),
        np.log(sorted_data[:,1]),
        line_width=0.5,
        color="green",
        label="Structure function (log/log)"
    )

    fit = gl.FitFromPolynomial(
        curve.create_slice_x(0, np.log(region_radius)),
        1,
        label="Fit (1st order polynomial)",
        line_width=1,
        color="orange",
    )

    return [curve, fit]

### Structure functions of all regions

In [3]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("Global region", None, 50),
    ("Diffuse region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 20),
    ("Central region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 10),
    ("Filament region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 6)
]

In [4]:
figs = []

for name, region, region_radius in regions:
    fig = gl.Figure(title=name, x_label="Lag logarithm", y_label="Structure function")
    fig.add_elements(*get_plottables(structure_function(turbulence.get_masked_region(region).data), region_radius))
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
multifig.save("figures/sh158/advanced_stats/structure_functions.pdf")